<a href="https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLabLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/shirooo39/MiXLab" target="_blank">
<img src="https://github.com/fluidicon.png" height="100px" align="left"></a>

# <font size=6><center>**Welcome to <font color=1DA261>Mi</font><font color=FFCf4C>XL</font><font color=4789F4>ab</font> Lite**</center></font>

<font size=4><center>This Notebook is a stripped down version of <a href="https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLab.ipynb" target="_blank">MixLab</a>.</center></font>

# <font size=6><center>✦ *Colab Stay Alive* ✦</center></font>
---

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] Colab Stay Alive</font>
# @markdown This cell runs a JS code that will automatically press the reconnect button when you got disconnected due to idle.
# ================================================================ #

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button"); 
     btn.click() 
     }
   
   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

clear_output()

# <font size=6><center>✦ *Mount/Unmount Google Drive* ✦</center></font>
---


In [ ]:
# ============================= FORM ============================= #
# @markdown <h3>← Mount/Unmount Google Drive</h3>
# @markdown <p>This cell will mount/unmount Google Drive to /content/drive/</p></br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
# ================================================================ #

from IPython.display import clear_output
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

clear_output()

In [ ]:
# @markdown <h3>← Force re-mount Google Drive</h3>

drive.mount("/content/drive", force_remount=True)

# <font size=6><center>✦ *qBittorrent* ✦</center></font>
---


In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] qBittorrent</font>
#QBITTORRENT_VARIANT = "official" #@param ["official", "unofficial"]
## @markdown ---
## @markdown <p>qBittorrent Default Credential</p>
## @markdown > Username: admin</br>Password: adminadmin
## @markdown ---
#USE_FREE_TOKEN = True  # @param {type:"boolean"}
#TOKEN = ""  # @param {type:"string"}
#PORT_FORWARD = "localhost" # @param ["ngrok", "localhost"]
USE_FREE_TOKEN = True
TOKEN = ""
PORT_FORWARD = "localhost"
# ================================================================ #

import os, psutil, time, urllib.request
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
                "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    checkAvailable,
    loadingAn,
    PortForward_wrapper,
    runSh,
    displayUrl,
    findProcess
)

#Note: need to locate where the WebUI is extracted into and then remove it
#      in order to use the proper WebUI for the Official or Unofficial version of qBittorrent
#runSh("rm -f /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list")
#runSh("rm -f /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list.save")
#runSh("rm -f /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list")
#runSh("rm -f /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list.save")
#runSh("rm -f /usr/bin/qbittorrent")
#runSh("rm -f /usr/bin/qbittorrent-nox")
#runSh("sudo apt-get purge --auto-remove qbittorrent-nox ")
#clear_output()

def addUtils():
    if not checkAvailable("/usr/local/sessionSettings"):
        runSh("mkdir -p -m 777 /usr/local/sessionSettings")
    if not checkAvailable("/content/upload.txt"):
        runSh("touch /content/upload.txt")
    if not checkAvailable("checkAptUpdate.txt", userPath=True):
        runSh("apt update -qq -y")
        runSh("apt-get install -y iputils-ping")

def configTimezone(auto=True):
    if checkAvailable("timezone.txt", userPath=True):
        return
    if not auto:
        runSh("sudo dpkg-reconfigure tzdata")
    else:
        runSh("sudo ln -fs /usr/share/zoneinfo/Asia/Ho_Chi_Minh /etc/localtime")
        runSh("sudo dpkg-reconfigure -f noninteractive tzdata")

def uploadQBittorrentConfig():
    if checkAvailable("updatedQBSettings.txt", userPath=True):
        return
    runSh(
        "mkdir -p -m 666 /content/qBittorrent /root/.qBittorrent_temp /root/.config/qBittorrent"
    )
    runSh(
        "wget -qq https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/configurations/qbittorrent/qBittorrent.conf \
            -O /root/.config/qBittorrent/qBittorrent.conf"
    )

def prepareSession():
    if checkAvailable("ready.txt", userPath=True):
        return
    else:
        addUtils()
        configTimezone()
        uploadQBittorrentConfig()

def installQBittorrent():
    if checkAvailable("/usr/bin/qbittorrent-nox"):
        return
    else:
#        if QBITTORRENT_VARIANT == "official":
            try:
#                if checkAvailable("/etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list"):
#                    runSh("rm /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list")
#                elif checkAvailable("/etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list.save"):
#                    runSh("rm /etc/apt/sources.list.d/poplite-ubuntu-qbittorrent-enhanced-bionic.list.save")
#                else:
                    runSh("sudo add-apt-repository ppa:qbittorrent-team/qbittorrent-stable")
                    runSh("sudo apt-get update")
                    runSh("sudo apt install qbittorrent-nox")
            except:
                raise Exception('Failed to install qBittorrent!')
#        elif QBITTORRENT_VARIANT == "unofficial":
#            try:
#                if checkAvailable("/etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list"):
#                    runSh("rm /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list")
#                elif checkAvailable("/etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list.save"):
#                    runSh("rm /etc/apt/sources.list.d/qbittorrent-team-ubuntu-qbittorrent-stable-bionic.list.save")
#                else:
#                    runSh("sudo add-apt-repository ppa:poplite/qbittorrent-enhanced")
#                    runSh("sudo apt-get update")
#                    runSh("sudo apt-get install qbittorrent-enhanced qbittorrent-enhanced-nox")
#            except:
#                raise Exception('Failed to install qBittorrent!')

def startQBService():
    prepareSession()
    installQBittorrent()
    if not findProcess("qbittorrent-nox", "-d --webui-port"):
        runSh(f"qbittorrent-nox -d --webui-port={QB_Port}")
        time.sleep(1)

QB_Port = 10001
loadingAn()
startQBService()

clear_output()

server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['qbittorrent', QB_Port, 'http']], 'sa', [f"{HOME}/.ngrok2/qbittorrent.yml", 4088]).start('qbittorrent', displayB=False)
displayUrl(server, pNamU='qBittorrent : ')